DENSITY FUNCTIONAL THEORY USING PYSCF

 
-the many body problem
-DFT
-Kohn sham DFT
-Hartree Fock DFT
-XC functionals and how to customise them 
-Examples: 1) pre defined variables 2) small molecule and customised xc 2) large molecule and customise all periodic boundary conditions.
-analysing the results

Introduction to DFT

Schrödinger's wave function is a useful mathematical construct for calculating the total energy of atoms and molecules, as well as other features. However, it can be very computationally expensive if used. (explain order of magnitude in Hamiltonian Eqn) Hence, the DFT is useful for calculating the ground state energy of larger molecules as it uses the electron density (experimentally observable) as well as wave functions to do so. 

#Kohn Shahm vs orbital free aproach (intro to xc functionals)
-it is a self consistent method.
Flowchart of how Kohn Sham : Initial guess of density eqn -> Calculate potentials(V) -> Solve many body Schrodingers eqns (wave fns) -> define density eqn (loop)


Building the Molecule 
-distance between atoms in Angstrom
- Basis (explain integration Grid)

The example shown below is for a water molecule 

In [ ]:
from pyscf import gto
from pyscf import dft

mol = gto.M(
    atom = '''
    O  0.   0.       0.
    H  0.   -0.757   0.587
    H  0.   0.757    0.587 ''',
    basis = 'ccpvdz')

Exchange Correlation Functional 
-various xcs in pyscf and how to pick the right one based on electron-electron and electron-nuclie interactions. 
- customisable xc 


Simple Example 1- Hydrogen Fluoride:

This example shows how to calculate the molecular orbital energy using the Kohn Shahm Density Functional theory. Here we are using a pre defined basis and xc functional known as the 

In [1]:
import pyscf
mol = pyscf.M(
    atom = 'H 0 0 0; F 0 0 1.1', 
    basis = '631g',
    symmetry = True,
)
mf = mol.KS()
mf.xc = 'pbe'
mf.kernel()
mf.analyze()

converged SCF energy = -100.291332797603
Wave-function symmetry = Coov
occupancy for each irrep:     A1  E1x  E1y
                               3    1    1
**** MO energy ****
MO #1 (A1 #1), energy= -24.2331684679437 occ= 2
MO #2 (A1 #2), energy= -1.0272973505931 occ= 2
MO #3 (A1 #3), energy= -0.426131284510634 occ= 2
MO #4 (E1x #1), energy= -0.289938626183792 occ= 2
MO #5 (E1y #1), energy= -0.289938626183793 occ= 2
MO #6 (A1 #4), energy= -0.034956037753547 occ= 0
MO #7 (A1 #5), energy= 0.664674826089051 occ= 0
MO #8 (E1x #2), energy= 1.13057079824108 occ= 0
MO #9 (E1y #2), energy= 1.13057079824107 occ= 0
MO #10 (A1 #6), energy= 1.24759794402076 occ= 0
MO #11 (A1 #7), energy= 1.57280304530478 occ= 0
 ** Mulliken atomic charges  **
charge of  0H =      0.35670
charge of  1F =     -0.35670
Dipole moment(X, Y, Z, Debye):  0.00000,  0.00000, -2.17040


((array([6.36542640e-01, 6.75369939e-03, 1.99996811e+00, 1.91722885e+00,
         3.01323431e-03, 1.99977249e+00, 1.99977249e+00, 1.43594348e+00,
         2.27508940e-04, 2.27508940e-04, 5.49982191e-04]),
  array([ 0.35670366, -0.35670366])),
 array([ 0.       ,  0.       , -2.1704032]))

Result Analysis for example 1

As seen from the output, the molecular oribtal energies were calculated for the molecule. These MO energies were used to build the density function n0(r).
This was used to calculate the SCF energy , which is the lowest possible ground state energy. 
Along with that 
